In [1]:
from selenium import webdriver
import sqlite3
from bs4 import BeautifulSoup
import re

Створюємо базу даних, яка міститиме такі параметри:
- ID - номер замовлення
- Відправлення - місто з якого відправляється вантаж
- Прибуття - куди прямує вантаж
- Маса вантажу
- Відстань між точками відправлення і прибуття
- Вартість послуги
- Спосіб оплати - готівка (1), картка(0)
- Безготівковий р-нок (1), відсутній - 0
- ПДВ (1), без ПДВ - 0
- Час оплати: на ровантажені - 1, інше - 0
- Транспорт - вид транспорту

In [27]:
db_conection = sqlite3.connect('lardi.db')


sql_text = """
            CREATE TABLE Orders(
                ID TEXT,
                Відправлення geography,
                Прибуття geography,
                Маса FLOAT,
                Обєм FLOAT,
                Відстань FLOAT,
                Вартість FLOAT,
                Спосіб оплати INT,
                Безготівковий розрахунок INT,
                ПДВ INT,
                Час оплати INT,
                Транспорт TEXT
                )  """

curs = db_conection.cursor()

curs.executescript(sql_text)
db_conection.close()

In [13]:

def response(page):
    DRIVER_PATH = 'C:\Program Files (x86)\Google\Chrome\chromedriver'
    driver = webdriver.Chrome(DRIVER_PATH)
    driver.get(f'https://lardi-trans.com/uk/gruz/c640h640x{page}.html')

    html = driver.page_source
    driver.close()
    s = BeautifulSoup(html)
    all_found = s.find('div', class_='ps_search-result_container').find_all('div', class_='ps_search-result_data-item')
    #global id_num = all_found.find("div", class_='ps_data_wrapper').get('data-ps-id')
    return all_found

def id_num(sth_found):
    id_num = sth_found.find_all("div",class_='ps_data_wrapper')[0].get('data-ps-id')
    return id_num

def mass(sth_found):
    try:
        details = sth_found.find('div', class_='ps_data ps_search-result_data-cargo ps_data-cargo').text.replace("\n\n\n", ', ')[1:-2].replace('\n\n','; ').replace('\n', '; ')
        mass = re.search(r'\b\d+\.*\d т*', details).group(0).replace(' т', '')
    except:
        mass = None
    return mass

def volume(sth_found):
    try:
        details = sth_found.find('div', class_='ps_data ps_search-result_data-cargo ps_data-cargo').text.replace("\n\n\n", ', ')[1:-2].replace('\n\n','; ').replace('\n', '; ')
        volume = re.search(r'\b\d+\.*\d* м3', details).group(0).replace(' м3', '')
        print(details)
    except:
        volume = None
    return volume

def payment_data(sth_found, mass):
    pay_dict = {}
    info = sth_found.find('span', class_='ps_data_payment_info').text + ', ' + sth_found.find('span', class_='ps_data_payment_details').text
    print(info)
    try:
        price = re.match(r'\d*', info).group(0)
        unit = re.search(r'грн\S*', info).group(0)
    except:
        price = None
        unit = None
    try:
        if unit == 'грн./т,':
            price = float(price)*float(mass[:-1])
        elif unit == 'грн.,':
            price = float(price)
        else:
            price = None
        pay_dict['price'] = price
    except:
        pay_dict['price'] = None
    
    if 'готів' in info:
        payment_method = 1
    elif 'карт' in info:
        payment_method = 0
    else:
        payment_method = None
    pay_dict['payment_method'] = payment_method
    
    if 'безгот' in info:
        cashless = 1
    else:
        cashless = 0
    pay_dict['cashless'] = cashless

    if 'ПДВ' in info:
        vat = 1
    else:
        vat = 0
    pay_dict['vat'] = vat

    if 'на розвант' in info:
        payment_time = 1
    else:
        payment_time = 0
    pay_dict['payment_time'] = payment_time
    return pay_dict

def from_(sth_found):
    from_ = (sth_found.find('div', class_ = 'ps_data_rout__from').find('span', class_='ps_data_town').text.replace('\n','') + \
             ',' + sth_found.find('div', class_ = 'ps_data_rout__from').find('span', class_='ps_data_region').text)
    return from_.split(',')[0]

def to_(sth_found):
    to_ = (sth_found.find('div', class_ = 'ps_data_rout__to').find('span', class_='ps_data_town').text.replace('\n','') + \
           ',' + sth_found.find('div', class_ = 'ps_data_rout__to').find('span', class_='ps_data_region').text)
    return to_.split(',')[0]

def transport(sth_found):
    transport = sth_found.find('span', class_='ps_data_transport').text.replace(' ','')
    return transport

def dist(sth_found):
    try:
        dist = sth_found.find("div",class_='ps_data_wrapper').find('div', class_='ps_data ps_search-result_data-counties ps_data-countries').find('span', class_='distance-tooltip tooltip-info').get('data-distance')
        dist = round(float(dist)/1000, 2)
    except:
        dist = None
    return dist



In [4]:
id_num_list = []

In [ ]:
    
conn = sqlite3.connect('lardi_db.db', check_same_thread = False)
for i in range(20):
    for page in range(25):
        print(page)
        try:
            all_found = response(page)
            for sth_found in all_found:   
                id_number = id_num(sth_found)
                if id_number not in id_num_list:
                    id_num_list.append(id_number) 
                    m = mass(sth_found)
                    payment = payment_data(sth_found, m)
                    price = payment['price']
                    payment_method = payment['payment_method']
                    cashless = payment['cashless']
                    vat = payment['vat']
                    payment_time = payment['payment_time']
                    text = f'("{id_number}", "{from_(sth_found)}", "{to_(sth_found)}",\
                                "{m}", "{volume(sth_found)}",\
                                "{dist(sth_found)}", "{price}",\
                                "{payment_method}", "{cashless}",\
                                "{vat}", "{payment_time}",\
                                "{transport(sth_found)}")'
                    sql_text = f"""INSERT INTO Orders VALUES {text}"""
                    curs = conn.cursor()
                    curs.execute(sql_text)
                    conn.commit()
        except:
            print('Problem(')

0


<ipython-input-13-1e7ec4610097>:3: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  driver = webdriver.Chrome(DRIVER_PATH)


запит вартості, 
запит вартості, готівк., на розвантаженні
 гофроящик, 0.5 т, 5 м3; завантаження: заднє; вигрузка: заднє
запит вартості, на розвантаженні
запит вартості, 
запит вартості, готівк., на розвантаженні
 гофроящик, 0.5 т, 5 м3; завантаження: заднє; вигрузка: заднє
запит вартості, 
3000 грн., карта, на розвантаженні
 вино на палетах, 2.4 т, 3 м3
9500 грн., комб., на розвантаженні
 хімія безпечна, 10.5 т, 30 м3; можливе довантаження; завантаження: заднє
запит вартості, готівк., на розвантаженні
4000 грн., карта, на розвантаженні
8400 грн., карта, на розвантаженні
запит вартості, 
запит вартості, безготівк., за оригіналами, з ПДВ
 тнв, 20 т, 86 м3; завантаження: заднє
запит вартості, карта, за оригіналами
 тнв, 20 т, 85 м3; завантаження: заднє
запит вартості, готівк., на розвантаженні
 гофрокартон, 1 т, 10 м3
6000 грн., карта, на розвантаженні
запит вартості, готівк., на розвантаженні
 гофроящик, 0.6 т, 6 м3; завантаження: заднє; вигрузка: заднє
7000 грн., готівк., на розвантаже

5300 грн., готівк., на розвантаженні
запит вартості, на розвантаженні, 50% передоплата
запит вартості, безготівк., на розвантаженні, 50% передоплата
запит вартості, безготівк., на розвантаженні, 50% передоплата
9
5000 грн., 
12500 грн., карта, на розвантаженні
 тара пластикова, 4 т, 90 - 120 м3
6000 грн., 
запит вартості, 
 поліграфія, 5 т, 8 м3; можливе довантаження
6000 грн., карта, на розвантаженні
 вторсырье, 4 т, 96 - 120 м3
6000 грн., готівк., на розвантаженні
запит вартості, 
3000 грн., готівк.
 МДФ, 5 т, 11.59 м3, ; Д. ; 2.8 м, ; Ш. ; 2.7 м, ; В. ; 2 м, 
3500 грн., готівк., на розвантаженні
5000 грн., ел. платіж, на розвантаженні
12000 грн., безготівк., з ПДВ
запит вартості, готівк.
запит вартості, 
 каністри, 3 т, 86 м3; завантаження: заднє, Д. ; 12.5 м, ; Ш. ; 2.4 м, ; В. ; 2.7 м, 
запит вартості, 
 щебень мелкой фракции, 22 т, 20 м3
6000 грн., готівк., на розвантаженні
запит вартості, 
 обладнання, 20 т, 20 м3; завантаження: верхнє
запит вартості, 
10
запит вартості, 
 стела

запит вартості, безготівк.
 цегла на палетах, 22 т, 86 м3; завантаження: бічне, верхнє; вигрузка: бічне, верхнє
20
запит вартості, з ПДВ
 тнв, 20 т, 86 м3; завантаження: бічне, з повною розтентовкою, заднє
2500 грн., карта, на розвантаженні
запит вартості, 
3500 грн., карта, на розвантаженні
запит вартості, безготівк.
 поліпропіленові блоки, 0.22 т, 9 м3
запит вартості, безготівк.
 поліпропіленові блоки, 0.22 т, 9 м3
1000 грн., готівк., на завантаженні
16500 грн., готівк., на розвантаженні
 торф навалом, 22 т, 60 м3
запит вартості, 
 мебель в пачках, 0.2 т, 3 м3
запит вартості, 
запит вартості, 
9000 грн., карта
16500 грн., готівк., на розвантаженні
 торф навалом, 22 т, 60 м3
7000 грн., карта
3500 грн., карта, на розвантаженні
900 грн., карта
 хозтовары, 0.7 т, 3.5 м3
запит вартості, готівк., на розвантаженні
 пластиковые изделия, 15 т, 75 м3
запит вартості, безготівк., за оригіналами, з ПДВ
 вода на палетах, 22 т, 86 м3; завантаження: заднє; вигрузка: заднє
900 грн., карта
 хозтовары,

...

запит вартості, карта, на розвантаженні
 уголь древесный в мешках, 16 - 22 т, 86 - 120 м3; 2 а/м разово; завантаження: заднє
запит вартості, карта, на розвантаженні
 уголь древесный в мешках, 5 т, 46 - 60 м3; завантаження: заднє
запит вартості, готівк.
12500 грн., готівк., на розвантаженні
запит вартості, готівк., на розвантаженні
 ліс-кругляк, 21 т, 22 м3
3
450 грн., безготівк., на розвантаженні, з ПДВ
запит вартості, готівк., на розвантаженні
запит вартості, 
 тнв, 13 т, 86 м3; завантаження: заднє; вигрузка: заднє
запит вартості, готівк., на розвантаженні
6400 грн., безготівк., з ПДВ
 напої на палетах, 21.6 т, 86 м3; завантаження: заднє
3500 грн., готівк.
 тнв, 0.5 т, 20 м3
2000 грн., готівк., на розвантаженні
 субстрат на палетах, 4 т, 10 м3
запит вартості, готівк., на розвантаженні
4500 грн., карта, на розвантаженні
 тнв, 5 т, 55 м3; завантаження: бічне
10500 грн., 
 папір в рулонах, 21 т, 86 м3; 1 а/м; завантаження: заднє
запит вартості, 
 тнв, 20 т, 86 м3; завантаження: заднє
зап

запит вартості, 
 тнв, 19 т, 86 м3; завантаження: заднє; вигрузка: заднє
запит вартості, 
 тнв, 19 т, 86 м3; завантаження: заднє; вигрузка: заднє
запит вартості, 
 тнв, 19 т, 86 м3; завантаження: заднє; вигрузка: заднє
запит вартості, 
запит вартості, 
запит вартості, готівк.
 вода на палетах, 22 т, 86 м3
11700 грн., готівк., на розвантаженні
 стройматериали, 20 т, 86 м3; завантаження: бічне, заднє; вигрузка: бічне, заднє
5600 грн., карта, на розвантаженні
 мінеральні добрива, 22 т, 86 м3; завантаження: бічне
5000 грн., 
запит вартості, карта, на розвантаженні
запит вартості, готівк., на розвантаженні
5000 грн., 
12
запит вартості, 
 палети порожні, 16 т, 86 м3
запит вартості, безготівк., з ПДВ
 утеплювач, 13 т, 86 м3; завантаження: заднє; вигрузка: бічне
запит вартості, 
 палети порожні, 16 т, 86 м3
запит вартості, безготівк., з ПДВ
 тнв, 20 т, 84 м3; завантаження: заднє
запит вартості, безготівк.
запит вартості, 
 палети порожні, 16 т, 86 м3
15000 грн., готівк., на розвантаженні
1500

21
запит вартості, 
9500 грн., карта, на розвантаженні
 вторсировина, 6 т, 105 - 120 м3; завантаження: верхнє, заднє
9000 грн., карта, на розвантаженні
 утеплювач, 1.8 т, 80 м3; завантаження: заднє
10000 грн., карта, на розвантаженні
 утеплювач, 2 т, 100 м3; завантаження: заднє
запит вартості, готівк., на розвантаженні
11000 грн., безготівк.
 гранули у біг-бегах, 20 т, 20 м3
9000 грн., готівк.
12000 грн., 
 пляшка на палетах, 15 т, 41 м3
15000 грн., готівк., на розвантаженні
7500 грн., готівк., на розвантаженні
5000 грн., готівк., на розвантаженні
8000 грн., 
20000 грн., безготівк.
9000 грн., 
 брикети, 13 т, 17 м3
запит вартості, 
 каністри, 3 т, 86 м3; завантаження: заднє; вигрузка: заднє, Д. ; 12.5 м, ; Ш. ; 2.4 м, ; В. ; 2.7 м, 
запит вартості, безготівк.
 тнв, 21 т, 90 м3; завантаження: бічне, В. ; 2.7 м, 
2000 грн., карта
запит вартості, з ПДВ
 металл, 8 т, 8 м3
запит вартості, 
 каністри, 3 т, 86 м3; завантаження: заднє; вигрузка: заднє, Д. ; 12.5 м, ; Ш. ; 2.4 м, ; В. ; 2.7 м, 

12000 грн., готівк., на розвантаженні
2500 грн., готівк., на розвантаженні
17000 грн., готівк., на розвантаженні
600 грн./т, готівк., на розвантаженні
600 грн./т, готівк., на розвантаженні
12000 грн., карта, на розвантаженні
11700 грн., безготівк., з ПДВ
6800 грн., готівк.
18000 грн., готівк., на розвантаженні
11
запит вартості, за оригіналами
 тнв, 22 т, 86 м3; завантаження: заднє
700 грн., готівк., на розвантаженні
 мебель в пачках, 0.18 т, 0.7 м3, ; Д. ; 2 м, 
12000 грн., готівк., на розвантаженні
запит вартості, безготівк.
7600 грн., готівк., на розвантаженні
5000 грн., готівк., на розвантаженні
8000 грн., готівк., на розвантаженні
1500 грн., готівк., на розвантаженні
1500 грн., готівк., на розвантаженні
2700 грн., готівк., на розвантаженні
8000 грн., готівк., на розвантаженні
запит вартості, 
700 грн., готівк., на розвантаженні
 мебель в пачках, 0.18 т, 0.7 м3, ; Д. ; 2 м, 
5400 грн., готівк., на розвантаженні
запит вартості, готівк., на розвантаженні
 мебель в коробках, 0.55 т, 7

14000 грн., 
6000 грн., карта, на розвантаженні
запит вартості, готівк., на розвантаженні
14000 грн., 
запит вартості, карта
 вода, 21.6 т, 86 м3; завантаження: заднє
1500 грн., 
2800 грн., готівк., на розвантаженні
 кава, 1 - 1.1 т, 6 - 7 м3; можливе довантаження; 1 а/м; завантаження: бічне, заднє; вигрузка: бічне, заднє
2700 грн., карта
 тнп, 3 т, 3 м3; завантаження: бічне
5500 грн., готівк., на розвантаженні
14000 грн., 
21
запит вартості, безготівк.
13000 грн., безготівк., з ПДВ
запит вартості, 
запит вартості, 
запит вартості, 
3000 грн., готівк., на розвантаженні
 тнв, 2 - 3 т, 86 - 120 м3
запит вартості, 
10780 грн., безготівк., з ПДВ
18000 грн., готівк., на розвантаженні
запит вартості, готівк., на розвантаженні
запит вартості, 
 ткань в рулонах, 0.5 т, 9 м3; можливе довантаження
1000 грн./т, безготівк., з ПДВ
 дробленая шелуха подсолничника, 18 - 25 т, 70 - 120 м3
900 грн./т, карта
 дробленая шелуха подсолничника, 19 - 25 т, 70 - 100 м3
800 грн./т, безготівк., з ПДВ
 топливная

In [11]:
len(id_num_list)

4640

In [31]:
all_found = response(1)
pay = payment_data(all_found[0], mass)

<ipython-input-23-8a9505690a40>:3: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  driver = webdriver.Chrome(DRIVER_PATH)


In [34]:
pay = payment_data(all_found[0], mass)
print(pay['vat'])

0


In [16]:
import pandas as pd
conn = sqlite3.connect('lardi_db.db', check_same_thread = False)
df = pd.read_sql('SELECT * FROM Orders', conn)
id_num_list = list(df['ID'])
conn.close()
df

,ID,Відправлення,Прибуття,Маса,Обєм,Відстань,Вартість,Спосіб,Безготівковий,ПДВ,Час,Транспорт
0,415071261,Хмельницький,Дніпро,3.5,None,683.86,4300.0,1,0,0,1,Крита
1,415026699,Путивль,Київ,22.0,None,305.75,7700.0,1,0,0,1,Тент
2,415688239,Городенка,Гоща,2.5,40.0,303.8,None,None,0,0,0,Тент
3,415688237,Кропивницький (Кіровоград),Миколаїв,20.0,None,181.33,14600.0,1,1,1,0,Зерновоз
4,415688235,Тернопіль,Львів,20.0,25.0,128.71,3000.0,None,0,0,1,Тент
...,...,...,...,...,...,...,...,...,...,...,...,...
19122,416006507,Хуст,Харьков,13.0,17.0,None,9000.0,None,0,0,0,Крита
19123,414458021,Смолянка,Киевмлын,200.0,None,None,None,None,0,0,0,Зерновоз
19124,416006501,Київ,Охтирка,21.0,None,363.69,None,1,1,0,0,Тент
19125,414458601,Козинці,Молодіжне,75.0,None,504.15,None,None,0,0,0,Зерновоз


In [17]:
import numpy as np
DF = df.copy()
df = df.replace(to_replace='None', value=np.nan)
df = df.dropna()
print(df)
df = df.drop_duplicates()
df


              ID              Відправлення       Прибуття  Маса   Обєм  \
6      415688231                 Тернопіль          Львів  20.0   30.0   
7      415684967                     Мошни        Черкаси  30.0   30.0   
9      415684155                  Єгорівка          Липці  16.0   12.0   
10     415684021                  Єгорівка       Близнюки  19.0   14.0   
11     415684071                  Єгорівка      Калинівка  21.0   17.0   
...          ...                       ...            ...   ...    ...   
19043  411543983  Чорноморськ (Іллічівськ)       Ватутіне  19.0  100.0   
19060  416005949                  Березове     Олексіївка  22.0   60.0   
19086  416006175                   Черкаси         Бахмач   0.2    4.5   
19099  276921500      Кам'янка-Дніпровська  Великі Копані  34.0   34.0   
19115  415969901                     Луцьк        Вінниця  22.0   86.0   

       Відстань  Вартість  Спосіб  Безготівковий  ПДВ  Час  Транспорт  
6        128.71    3000.0     0.0      

,ID,Відправлення,Прибуття,Маса,Обєм,Відстань,Вартість,Спосіб,Безготівковий,ПДВ,Час,Транспорт
6,415688231,Тернопіль,Львів,20.0,30.0,128.71,3000.0,0.0,0,0,0,Тент
7,415684967,Мошни,Черкаси,30.0,30.0,31.04,700.0,1.0,1,0,1,Крита
9,415684155,Єгорівка,Липці,16.0,12.0,722.65,9800.0,1.0,0,0,1,Тент
10,415684021,Єгорівка,Близнюки,19.0,14.0,621.84,11300.0,1.0,0,0,1,Тент
11,415684071,Єгорівка,Калинівка,21.0,17.0,419.19,8500.0,1.0,0,0,1,Тент
...,...,...,...,...,...,...,...,...,...,...,...,...
19043,411543983,Чорноморськ (Іллічівськ),Ватутіне,19.0,100.0,368.77,17100.0,0.0,0,0,0,Зерновоз
19060,416005949,Березове,Олексіївка,22.0,60.0,434.37,16000.0,1.0,0,0,1,Самоскид
19086,416006175,Черкаси,Бахмач,0.2,4.5,270.34,800.0,1.0,0,0,1,Цільномет
19099,276921500,Кам'янка-Дніпровська,Великі Копані,34.0,34.0,197.99,11500.0,1.0,0,0,1,Крита


In [18]:
df

,ID,Відправлення,Прибуття,Маса,Обєм,Відстань,Вартість,Спосіб,Безготівковий,ПДВ,Час,Транспорт
6,415688231,Тернопіль,Львів,20.0,30.0,128.71,3000.0,0.0,0,0,0,Тент
7,415684967,Мошни,Черкаси,30.0,30.0,31.04,700.0,1.0,1,0,1,Крита
9,415684155,Єгорівка,Липці,16.0,12.0,722.65,9800.0,1.0,0,0,1,Тент
10,415684021,Єгорівка,Близнюки,19.0,14.0,621.84,11300.0,1.0,0,0,1,Тент
11,415684071,Єгорівка,Калинівка,21.0,17.0,419.19,8500.0,1.0,0,0,1,Тент
...,...,...,...,...,...,...,...,...,...,...,...,...
19043,411543983,Чорноморськ (Іллічівськ),Ватутіне,19.0,100.0,368.77,17100.0,0.0,0,0,0,Зерновоз
19060,416005949,Березове,Олексіївка,22.0,60.0,434.37,16000.0,1.0,0,0,1,Самоскид
19086,416006175,Черкаси,Бахмач,0.2,4.5,270.34,800.0,1.0,0,0,1,Цільномет
19099,276921500,Кам'янка-Дніпровська,Великі Копані,34.0,34.0,197.99,11500.0,1.0,0,0,1,Крита


In [19]:

df_old = pd.read_csv('lardi_data.csv')
df = pd.concat([df_old, df])

In [20]:
df_old

,Unnamed: 0,ID,Відправлення,Прибуття,Маса,Обєм,Відстань,Вартість,Спосіб,Безготівковий,ПДВ,Час,Транспорт
0,0,415087057,Чорноморськ (Іллічівськ),Новомосковськ,15.0,100.0,520.48,15000.0,0.0,0,0,1,Зерновоз
1,1,414586695,Короп,Кривий Ріг,22.0,86.0,525.27,13000.0,1.0,0,0,1,Тент
2,2,414613875,Кролевець,Кривий Ріг,22.0,86.0,532.79,13000.0,1.0,0,0,1,Тент
3,3,415087239,Кривий Ріг,Дніпро,97.0,86.0,146.01,6000.0,1.0,1,1,0,Тент
4,4,415087407,Хацьки,Красногірка,10.2,45.0,294.13,7500.0,1.0,1,1,0,Тент
...,...,...,...,...,...,...,...,...,...,...,...,...,...
9905,19043,411543983,Чорноморськ (Іллічівськ),Ватутіне,19.0,100.0,368.77,17100.0,0.0,0,0,0,Зерновоз
9906,19060,416005949,Березове,Олексіївка,22.0,60.0,434.37,16000.0,1.0,0,0,1,Самоскид
9907,19086,416006175,Черкаси,Бахмач,0.2,4.5,270.34,800.0,1.0,0,0,1,Цільномет
9908,19099,276921500,Кам'янка-Дніпровська,Великі Копані,34.0,34.0,197.99,11500.0,1.0,0,0,1,Крита


In [21]:
df

,Unnamed: 0,ID,Відправлення,Прибуття,Маса,Обєм,Відстань,Вартість,Спосіб,Безготівковий,ПДВ,Час,Транспорт
0,0.0,415087057,Чорноморськ (Іллічівськ),Новомосковськ,15.0,100.0,520.48,15000.0,0.0,0,0,1,Зерновоз
1,1.0,414586695,Короп,Кривий Ріг,22.0,86.0,525.27,13000.0,1.0,0,0,1,Тент
2,2.0,414613875,Кролевець,Кривий Ріг,22.0,86.0,532.79,13000.0,1.0,0,0,1,Тент
3,3.0,415087239,Кривий Ріг,Дніпро,97.0,86.0,146.01,6000.0,1.0,1,1,0,Тент
4,4.0,415087407,Хацьки,Красногірка,10.2,45.0,294.13,7500.0,1.0,1,1,0,Тент
...,...,...,...,...,...,...,...,...,...,...,...,...,...
19043,NaN,411543983,Чорноморськ (Іллічівськ),Ватутіне,19.0,100.0,368.77,17100.0,0.0,0,0,0,Зерновоз
19060,NaN,416005949,Березове,Олексіївка,22.0,60.0,434.37,16000.0,1.0,0,0,1,Самоскид
19086,NaN,416006175,Черкаси,Бахмач,0.2,4.5,270.34,800.0,1.0,0,0,1,Цільномет
19099,NaN,276921500,Кам'янка-Дніпровська,Великі Копані,34.0,34.0,197.99,11500.0,1.0,0,0,1,Крита


In [22]:
df.drop(columns=['Unnamed: 0'], inplace=True)

In [23]:
df = df.drop_duplicates()
df

,ID,Відправлення,Прибуття,Маса,Обєм,Відстань,Вартість,Спосіб,Безготівковий,ПДВ,Час,Транспорт
0,415087057,Чорноморськ (Іллічівськ),Новомосковськ,15.0,100.0,520.48,15000.0,0.0,0,0,1,Зерновоз
1,414586695,Короп,Кривий Ріг,22.0,86.0,525.27,13000.0,1.0,0,0,1,Тент
2,414613875,Кролевець,Кривий Ріг,22.0,86.0,532.79,13000.0,1.0,0,0,1,Тент
3,415087239,Кривий Ріг,Дніпро,97.0,86.0,146.01,6000.0,1.0,1,1,0,Тент
4,415087407,Хацьки,Красногірка,10.2,45.0,294.13,7500.0,1.0,1,1,0,Тент
...,...,...,...,...,...,...,...,...,...,...,...,...
19043,411543983,Чорноморськ (Іллічівськ),Ватутіне,19.0,100.0,368.77,17100.0,0.0,0,0,0,Зерновоз
19060,416005949,Березове,Олексіївка,22.0,60.0,434.37,16000.0,1.0,0,0,1,Самоскид
19086,416006175,Черкаси,Бахмач,0.2,4.5,270.34,800.0,1.0,0,0,1,Цільномет
19099,276921500,Кам'янка-Дніпровська,Великі Копані,34.0,34.0,197.99,11500.0,1.0,0,0,1,Крита


In [24]:
df.to_csv('lardi_data.csv')

In [25]:
df = pd.read_csv('lardi_data.csv')
df = df.drop(columns=['Unnamed: 0'])
df

,ID,Відправлення,Прибуття,Маса,Обєм,Відстань,Вартість,Спосіб,Безготівковий,ПДВ,Час,Транспорт
0,415087057,Чорноморськ (Іллічівськ),Новомосковськ,15.0,100.0,520.48,15000.0,0.0,0,0,1,Зерновоз
1,414586695,Короп,Кривий Ріг,22.0,86.0,525.27,13000.0,1.0,0,0,1,Тент
2,414613875,Кролевець,Кривий Ріг,22.0,86.0,532.79,13000.0,1.0,0,0,1,Тент
3,415087239,Кривий Ріг,Дніпро,97.0,86.0,146.01,6000.0,1.0,1,1,0,Тент
4,415087407,Хацьки,Красногірка,10.2,45.0,294.13,7500.0,1.0,1,1,0,Тент
...,...,...,...,...,...,...,...,...,...,...,...,...
10272,411543983,Чорноморськ (Іллічівськ),Ватутіне,19.0,100.0,368.77,17100.0,0.0,0,0,0,Зерновоз
10273,416005949,Березове,Олексіївка,22.0,60.0,434.37,16000.0,1.0,0,0,1,Самоскид
10274,416006175,Черкаси,Бахмач,0.2,4.5,270.34,800.0,1.0,0,0,1,Цільномет
10275,276921500,Кам'янка-Дніпровська,Великі Копані,34.0,34.0,197.99,11500.0,1.0,0,0,1,Крита


In [28]:
conn = sqlite3.connect('lardi.db', check_same_thread = False)
df.to_sql('lardi.db', conn)